# Appendix 3: reweighting existing samples

Johann Brehmer, Felix Kling, Irina Espejo, and Kyle Cranmer 2018-2019

What if you already have a bunch of generated events, either simulated with MadMiner with a different benchmark setup, or with stand-alone MadGraph? MadMiner now lets you add the missing event weights to these files.

## 0. Preparations

In [1]:
import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline

from madminer.core import MadMiner

MadMiner uses the Python `logging` module to provide additional information and debugging output. You can choose how much of this output you want to see by switching the level in the following lines to `logging.DEBUG` or `logging.WARNING`.

In [2]:
# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## 1. Original event sample

We'll start with the event sample from the main tutorial, and assume we forgot to add one benchmark we *really* care about before starting the event generation. Let's try to add this benchmark (and the corresponding weights) after generating the events!

We will assume that you have generated events according to the following settings, please adapt them if necessary:

In [3]:
mg_process_dir = "mg_processes/signal1"
run_name = "run_01"
sampling_benchmark = "sm"

## 2. Load setup and add new benchmark

In [4]:
miner = MadMiner()
miner.load("data/setup.h5")
miner.add_benchmark({"CWL2": 10.0, "CPWL2": 8.0}, "oh_shit_we_forgot_this")
miner.save("data/setup_with_extra_benchmark.h5")

07:15 madminer.core        INFO    Found 2 parameters:
07:15 madminer.core        INFO       CWL2 (LHA: dim6 2, maximal power in squared ME: (2,), range: (-20.0, 20.0))
07:15 madminer.core        INFO       CPWL2 (LHA: dim6 5, maximal power in squared ME: (2,), range: (-20.0, 20.0))
07:15 madminer.core        INFO    Found 6 benchmarks:
07:15 madminer.core        INFO       sm: CWL2 = 0.00e+00, CPWL2 = 0.00e+00
07:15 madminer.core        INFO       w: CWL2 = 15.20, CPWL2 = 0.10
07:15 madminer.core        INFO       neg_w: CWL2 = -1.54e+01, CPWL2 = 0.20
07:15 madminer.core        INFO       ww: CWL2 = 0.30, CPWL2 = 15.10
07:15 madminer.core        INFO       neg_ww: CWL2 = 0.40, CPWL2 = -1.53e+01
07:15 madminer.core        INFO       morphing_basis_vector_5: CWL2 = -1.68e+01, CPWL2 = -1.72e+01
07:15 madminer.core        INFO    Found morphing setup with 6 components
07:15 madminer.core        INFO    Did not find systematics setup.
07:15 madminer.core        INFO    Added benchmark oh_s

This doesn't change the morphing setup (which always uses the first benchmarks, i.e. the ones that were already defined in the MadMiner file).

## 3. Prepare reweighting

Here's the crucial step. `reweight_benchmarks` is a list of all the benchmarks that the sample will be reweighted to -- this should *not* include the benchmarks for which the sample already contains weights.

In [5]:
miner.reweight_existing_sample(
    mg_process_directory=mg_process_dir,
    run_name=run_name,
    param_card_template_file="Cards/param_card_template.dat",
    sample_benchmark=sampling_benchmark,
    reweight_benchmarks=["oh_shit_we_forgot_this"],
)

07:15 madminer.core        INFO    Reweighting setup
07:15 madminer.core        INFO      Originally sampled from benchmark: sm
07:15 madminer.core        INFO      Now reweighting to benchmarks:     ['oh_shit_we_forgot_this']
07:15 madminer.core        INFO      Reweight card:                     /madminer/cards/reweight_card_reweight.dat
07:15 madminer.core        INFO      Log file:                          reweight.log
07:15 madminer.core        INFO    Creating param and reweight cards in mg_processes/signal1
07:15 madminer.utils.inter INFO    Starting reweighting of an existing sample in mg_processes/signal1
07:16 madminer.core        INFO    Finished running reweighting! Please check that events were succesfully reweighted in the following folder:

 mg_processes/signal1/Events/run_01 


